In [1]:
import pandas as pd
import numpy as np
import torch
import os

from transformers import AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from mkdataset import EmotionDataset, TestDataset
from datasets import load_metric, load_dataset
from classifier import RobertaForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from utils import set_allseed
import warnings

In [2]:
seed = 777
batch_size = 32
save_steps = 103
set_allseed(seed)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

gpu = torch.device("cuda")
cpu = torch.device("cpu")
warnings.filterwarnings(action='ignore') 

In [3]:
def compute_metrics(pred):
    f1 = load_metric("f1")
    references = pred.label_ids
    predictions = pred.predictions.argmax(axis=1)
    metric = f1.compute(predictions=predictions, references=references, average="micro")
    return metric

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df = train_df[["문장","극성"]]

In [6]:
kfold_function = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
type_config = AutoConfig.from_pretrained("klue/roberta-base")
type_config.num_labels = len(train_df.극성.value_counts())

In [7]:
training_args = TrainingArguments(
    output_dir="./output_type",
    seed=seed,
    save_total_limit=2,
    save_steps = save_steps,
    num_train_epochs = 5,
    learning_rate= 1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    weight_decay=1e-4,
    logging_dir="./logs",
    logging_steps = save_steps,
    evaluation_strategy = "steps",
    metric_for_best_model = "eval_f1",
    eval_steps = save_steps,
    load_best_model_at_end=True,
)
test_dataset = TestDataset(data=test_df, tokenizer=tokenizer)

In [8]:
logit = 0
for i, (train_index, test_index) in enumerate(kfold_function.split(train_df["문장"],train_df["극성"])):
    model = RobertaForSequenceClassification.from_pretrained("klue/roberta-base", config=type_config)
    train_corpus, valid_corpus = train_df["문장"][train_index], train_df["문장"][test_index]
    train_label, valod_label = train_df["극성"][train_index], train_df["극성"][test_index]
    fold_train = pd.concat([train_corpus, train_label], axis =1)
    fold_valid = pd.concat([valid_corpus, valod_label], axis =1)
    train_dataset = EmotionDataset(data=fold_train, tokenizer=tokenizer)
    valid_dataset = EmotionDataset(data=fold_valid, tokenizer=tokenizer)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    )

    trainer.train()

    logit += trainer.predict(test_dataset).predictions / 5

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['lstm.weight_ih_l1_reverse', 'lstm.bias_hh_l1_reverse', 'robert

  0%|          | 0/515 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 3309
  Batch size = 32


{'loss': 0.1503, 'learning_rate': 8e-05, 'epoch': 1.0}


  0%|          | 0/104 [00:00<?, ?it/s]

...

Saving model checkpoint to ./output_type\checkpoint-515
Configuration saved in ./output_type\checkpoint-515\config.json


{'eval_loss': 0.1144118383526802, 'eval_f1': 0.9746070133010882, 'eval_runtime': 9.6107, 'eval_samples_per_second': 344.201, 'eval_steps_per_second': 10.821, 'epoch': 5.0}


Model weights saved in ./output_type\checkpoint-515\pytorch_model.bin
Deleting older checkpoint [output_type\checkpoint-412] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./output_type\checkpoint-309 (score: 0.9758162031438936).
***** Running Prediction *****
  Num examples = 7090
  Batch size = 32


{'train_runtime': 1093.3356, 'train_samples_per_second': 60.517, 'train_steps_per_second': 0.471, 'train_loss': 0.07015352804683944, 'epoch': 5.0}


  0%|          | 0/222 [00:00<?, ?it/s]

In [9]:
result = pd.DataFrame(logit.argmax(axis=1).tolist(), columns=["emotion"])
test_df_result = pd.concat([test_df,result],axis=1, ignore_index=True)
test_df_result.to_csv("result_emotion.csv")